# Generate recordings with custom-defined spike trains

In this notebook, we sill show how to use custom spike trains as input to the simulation. 
We will use simple spike trains, but the user could use network simulations (e.g. using BRIAN or NEST).

In [ ]:
import MEArec as mr
import matplotlib.pyplot as plt
import neo
import quantities as pq
import numpy as np
import os
import numpy as np
%matplotlib notebook

# Generate custom spike trains

Let's generate 4 custom spike trains: 
- 2 bursting neurons as excitatory cells
- 2 constant neurons as inhibitory cells

We need to create a list of `neo.SpikeTrain` objects. We can annotate the spike trains with the `cell_type` keyword to tell the simulator whether a cell is excitatory (E) or inhibitory (I).

In [ ]:
duration = 10  # s
t_start = 0
burst_duration = 0.5
n_spikes_per_burst = 5
inter_burst = 2

spike_times = []
cell_types = []
for i in range(2):
    if i == 0:        
        burst_start = 0.1
    else:
        burst_start = 0.65
    st = np.array([])
    while burst_start < duration - burst_duration:
        burst = np.linspace(burst_start, burst_start + burst_duration, n_spikes_per_burst)
        st = np.concatenate((st, burst))
        burst_start = burst_start + burst_duration + inter_burst
    spike_times.append(st)
    cell_types.append('E')
    
for i in range(2):
    if i == 0:
        spike_train_start = 0.1
        spike_train_stop = duration - 0.2
    else:
        spike_train_start = 0.25
        spike_train_stop = duration - 0.1
    st = np.linspace(spike_train_start, spike_train_stop, 20)
    spike_times.append(st)
    cell_types.append('I')
    
# create neo spiketrains
spike_trains = []
for i, st in enumerate(spike_times):
    spiketrain = neo.SpikeTrain(times=st * pq.s, t_start=t_start*pq.s, t_stop=duration*pq.s)
    spiketrain.annotate(cell_type=cell_types[i])
    spike_trains.append(spiketrain)

### Create SpikeTrainGenerator object

Now we need to create a MEArec `SpikeTrainGenerator` object, which will be used in the recordings generation phase. We can also use the MEArec API to plot the spike trains.

In [ ]:
spgen = mr.SpikeTrainGenerator(spiketrains=spike_trains)

In [ ]:
_ = mr.plot_rasters(spgen.spiketrains, cell_type=True)

# Generate templates

Let's now generate a few templates on a tetrode to build our recordings. If you have already simulated the templates or you have others, you can skip this section.

In [ ]:
template_params = mr.get_default_templates_params()
cell_folder = mr.get_default_cell_models_folder()

In [ ]:
template_params['probe'] = 'tetrode'
template_params['n'] = 5

In [ ]:
tempgen = mr.gen_templates(cell_models_folder=cell_folder, params=template_params, verbose=True)

In [ ]:
template_filename = 'templates_tetrodes.h5'
mr.save_template_generator(tempgen, template_filename)

# Generate recordings

We have to load the default parameters, so we can modify some of them.

In [ ]:
template_filename = 'templates_tetrodes.h5'

In [ ]:
rec_params = mr.get_default_recordings_params()
rec_params['recordings']['noise_level'] = 5
rec_params['recordings']['filter'] = False
rec_params['templates']['min_dist'] = 5

In [ ]:
recgen = mr.gen_recordings(params=rec_params, spgen=spgen, templates=template_filename, verbose=True)

In [ ]:
mr.plot_recordings(recgen, overlay_templates=True, lw=0.5)